In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import StringIO
import tkinter as tk
from tkinter import ttk, messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# URLs for the data files
URL_DUAL_FUEL_BACKGROUND = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/1.Dual_Fuel_Background_Data.csv"
URL_DUAL_FUEL_IMPACTS = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/2.Dual_Fuel_Impacts.csv"
URL_EMISSION_IMPACTS = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/3.Emission_Impacts.csv"
URL_ENGINE_DATA = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/4.Engine_Data.csv"
URL_ASSUMPTIONS = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/5.Assumptions.csv"
URL_EMISSIONS = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/6.Emission.csv"

# Load data from GitHub
def load_data(url):
    response = requests.get(url)
    return pd.read_csv(StringIO(response.text), sep=';', decimal=',')

# Load all datasets
dual_fuel_bg = load_data(URL_DUAL_FUEL_BACKGROUND)
dual_fuel_impacts = load_data(URL_DUAL_FUEL_IMPACTS)
emission_impacts = load_data(URL_EMISSION_IMPACTS)
engine_data = load_data(URL_ENGINE_DATA)
assumptions = load_data(URL_ASSUMPTIONS)
emissions = load_data(URL_EMISSIONS)

# Debug: Print column names to see what we're working with
print("Dual Fuel Background columns:", dual_fuel_bg.columns.tolist())
print("Dual Fuel Impacts columns:", dual_fuel_impacts.columns.tolist())
print("Engine Data columns:", engine_data.columns.tolist())

# Preprocess data
# Clean up the engine data
engine_data = engine_data.iloc[:, :7]
engine_data.columns = ['Engine Size', 'Max Volume Flow', 'Min Volume Flow', 
                       'Minimum Engine Load', 'Engine Efficiency', 'Oil Tank Volume', 'Production Factor']

# Extract assumptions
calorific_ch4 = assumptions.iloc[0, 0]
calorific_diesel = assumptions.iloc[1, 0]
annual_runtime = assumptions.iloc[2, 0]

# Clean up emissions data
emissions = emissions.iloc[:, :6]
emissions.columns = ['Engine Load', 'CO', 'NOx', 'Formaldehyd', 'CH4 slop', 'Particulate Matter']

# Create the main application class
class LoCaGasApp:
    def __init__(self, root):
        self.root = root
        self.root.title("LoCaGas - Dual Fuel Motor Environmental Impact Analysis")
        self.root.geometry("1200x800")
        
        # Initialize variables
        self.years = tk.IntVar(value=10)
        self.manual_input = tk.BooleanVar(value=False)
        self.initial_gas_volume = tk.DoubleVar(value=500.0)
        self.initial_methane_content = tk.DoubleVar(value=50.0)
        
        # Create manual input storage
        self.manual_gas_volumes = {}
        self.manual_methane_contents = {}
        
        # Create the main notebook (tabbed interface)
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Create tabs
        self.input_tab = ttk.Frame(self.notebook)
        self.results_tab = ttk.Frame(self.notebook)
        self.details_tab = ttk.Frame(self.notebook)
        self.calculations_tab = ttk.Frame(self.notebook)
        
        self.notebook.add(self.input_tab, text="Eingabe")
        self.notebook.add(self.results_tab, text="Ergebnisse")
        self.notebook.add(self.details_tab, text="Detailergebnisse")
        self.notebook.add(self.calculations_tab, text="Zwischenberechnungen")
        
        # Setup input tab
        self.setup_input_tab()
        
        # Setup results tab (will be populated after calculation)
        self.setup_results_tab()
        
        # Setup details tab (will be populated after calculation)
        self.setup_details_tab()
        
        # Setup calculations tab (will be populated after calculation)
        self.setup_calculations_tab()
    
    def setup_input_tab(self):
        # Input parameters frame
        input_frame = ttk.LabelFrame(self.input_tab, text="Eingabeparameter", padding=10)
        input_frame.pack(fill='x', padx=10, pady=10)
        
        # Years input
        ttk.Label(input_frame, text="Betrachtungszeitraum [Jahre]:").grid(row=0, column=0, sticky='w', pady=5)
        ttk.Spinbox(input_frame, from_=1, to=50, textvariable=self.years, width=10).grid(row=0, column=1, pady=5, padx=5)
        
        # Manual input checkbox
        ttk.Checkbutton(input_frame, text="Manuelle Eingabe", variable=self.manual_input, 
                       command=self.toggle_manual_input).grid(row=1, column=0, columnspan=2, sticky='w', pady=5)
        
        # Initial gas volume
        ttk.Label(input_frame, text="Gasmenge zu Beginn [m³/h]:").grid(row=2, column=0, sticky='w', pady=5)
        ttk.Entry(input_frame, textvariable=self.initial_gas_volume, width=10).grid(row=2, column=1, pady=5, padx=5)
        
        # Initial methane content
        ttk.Label(input_frame, text="Methangehalt zu Beginn [%]:").grid(row=3, column=0, sticky='w', pady=5)
        ttk.Entry(input_frame, textvariable=self.initial_methane_content, width=10).grid(row=3, column=1, pady=5, padx=5)
        
        # Calculate button
        ttk.Button(input_frame, text="Berechnen", command=self.calculate).grid(row=4, column=0, columnspan=2, pady=10)
        
        # Manual input frame (initially hidden)
        self.manual_frame = ttk.LabelFrame(self.input_tab, text="Manuelle Eingabe für jedes Jahr", padding=10)
        
    def toggle_manual_input(self):
        if self.manual_input.get():
            self.manual_frame.pack(fill='x', padx=10, pady=10)
            self.create_manual_inputs()
        else:
            self.manual_frame.pack_forget()
    
    def create_manual_inputs(self):
        # Clear existing manual inputs
        for widget in self.manual_frame.winfo_children():
            widget.destroy()
        
        # Create input fields for each year
        years = self.years.get()
        self.manual_gas_volumes = {}
        self.manual_methane_contents = {}
        
        for i in range(years):
            year = i + 1
            ttk.Label(self.manual_frame, text=f"Jahr {year}:").grid(row=i, column=0, padx=5, pady=2)
            
            # Gas volume input
            gas_var = tk.DoubleVar(value=self.initial_gas_volume.get() * (0.9 ** year))
            ttk.Label(self.manual_frame, text="Gasmenge [m³/h]:").grid(row=i, column=1, padx=5, pady=2)
            ttk.Entry(self.manual_frame, textvariable=gas_var, width=10).grid(row=i, column=2, padx=5, pady=2)
            self.manual_gas_volumes[year] = gas_var
            
            # Methane content input
            meth_var = tk.DoubleVar(value=self.initial_methane_content.get() * (0.9 ** year))
            ttk.Label(self.manual_frame, text="Methangehalt [%]:").grid(row=i, column=3, padx=5, pady=2)
            ttk.Entry(self.manual_frame, textvariable=meth_var, width=10).grid(row=i, column=4, padx=5, pady=2)
            self.manual_methane_contents[year] = meth_var
    
    def setup_results_tab(self):
        # Results will be populated after calculation
        self.results_frame = ttk.Frame(self.results_tab)
        self.results_frame.pack(fill='both', expand=True, padx=10, pady=10)
        
    def setup_details_tab(self):
        # Details will be populated after calculation
        self.details_frame = ttk.Frame(self.details_tab)
        self.details_frame.pack(fill='both', expand=True, padx=10, pady=10)
        
    def setup_calculations_tab(self):
        # Calculations will be populated after calculation
        self.calculations_frame = ttk.Frame(self.calculations_tab)
        self.calculations_frame.pack(fill='both', expand=True, padx=10, pady=10)
    
    def calculate(self):
        try:
            # Get input values
            years = self.years.get()
            manual_input = self.manual_input.get()
            initial_gas_volume = self.initial_gas_volume.get()
            initial_methane_content = self.initial_methane_content.get()
            
            # Calculate gas development over time
            gas_volumes = [initial_gas_volume]
            methane_contents = [initial_methane_content]
            
            if manual_input:
                for year in range(1, years + 1):
                    gas_volumes.append(self.manual_gas_volumes[year].get())
                    methane_contents.append(self.manual_methane_contents[year].get())
            else:
                for year in range(1, years + 1):
                    gas_volumes.append(gas_volumes[-1] * 0.9)
                    methane_contents.append(methane_contents[-1] * 0.9)
            
            # Create a DataFrame for the gas development
            gas_df = pd.DataFrame({
                'Jahr': range(1, years + 1),
                'Gasmenge [m³/h]': gas_volumes[1:],
                'Methangehalt [%]': methane_contents[1:]
            })
            
            # Function to calculate engine load for a given engine size
            def calculate_engine_load(engine_size, gas_volume, methane_content):
                # Get diesel consumption for this engine size at 100% load (simplified)
                # Find the correct column name for diesel consumption
                diesel_col = None
                for col in dual_fuel_bg.columns:
                    if f'{engine_size}' in col and ('Diesel' in col or 'diesel' in col.lower()):
                        diesel_col = col
                        break
                
                if diesel_col is None:
                    raise ValueError(f"Diesel consumption column for engine size {engine_size} not found")
                
                diesel_cons = dual_fuel_bg[dual_fuel_bg.iloc[:, 0] == 100][diesel_col].values[0]
                
                # Get engine efficiency
                engine_eff = engine_data[engine_data['Engine Size'] == engine_size]['Engine Efficiency'].values[0] / 100
                
                # Calculate energy from methane
                methane_energy = (methane_content / 100) * gas_volume * calorific_ch4
                
                # Calculate energy from diesel
                diesel_energy = diesel_cons * calorific_diesel
                
                # Calculate total energy output
                total_energy = (methane_energy + diesel_energy) * engine_eff
                
                # Calculate engine load (simplified)
                engine_load = min(100, total_energy / engine_size * 100)
                
                return engine_load, diesel_cons
            
            # Determine the best engine size
            engine_sizes = [250, 500, 750, 1000]
            best_engine_size = None
            best_engine_load = 0
            
            for size in engine_sizes:
                engine_load, _ = calculate_engine_load(size, initial_gas_volume, initial_methane_content)
                
                # Check constraints
                min_flow = engine_data[engine_data['Engine Size'] == size]['Min Volume Flow'].values[0]
                max_flow = engine_data[engine_data['Engine Size'] == size]['Max Volume Flow'].values[0]
                min_load = engine_data[engine_data['Engine Size'] == size]['Minimum Engine Load'].values[0]
                
                if (min_flow <= initial_gas_volume <= max_flow) and (engine_load >= min_load) and (engine_load <= 100):
                    if engine_load > best_engine_load:
                        best_engine_load = engine_load
                        best_engine_size = size
            
            if best_engine_size is None:
                messagebox.showerror("Fehler", "Keine geeignete Motorgröße gefunden. Bitte überprüfen Sie die Eingabeparameter.")
                return
            
            # Get data for the selected engine size
            selected_engine = engine_data[engine_data['Engine Size'] == best_engine_size]
            production_factor = selected_engine['Production Factor'].values[0]
            oil_tank_volume = selected_engine['Oil Tank Volume'].values[0]
            
            # Get diesel consumption for the selected engine
            diesel_col = None
            for col in dual_fuel_bg.columns:
                if f'{best_engine_size}' in col and ('Diesel' in col or 'diesel' in col.lower()):
                    diesel_col = col
                    break
            
            if diesel_col is None:
                raise ValueError(f"Diesel consumption column for engine size {best_engine_size} not found")
            
            diesel_consumption = dual_fuel_bg[dual_fuel_bg.iloc[:, 0] == 100][diesel_col].values[0]
            
            # Calculate environmental impacts for each year
            impact_categories = dual_fuel_impacts.columns[1:]
            
            # Initialize results
            results = {
                'year': [],
                'engine_production': {cat: 0 for cat in impact_categories},
                'oil_consumption': {cat: 0 for cat in impact_categories},
                'diesel_consumption': {cat: 0 for cat in impact_categories},
                'maintenance': {cat: 0 for cat in impact_categories},
                'emissions': {cat: 0 for cat in impact_categories}
            }
            
            # Add engine production impacts (one-time)
            for cat in impact_categories:
                engine_prod_value = dual_fuel_impacts[dual_fuel_impacts['Impact Category'] == 'Engine Production'][cat].values[0]
                results['engine_production'][cat] = engine_prod_value * production_factor
            
            # Calculate impacts for each year
            for year in range(1, years + 1):
                results['year'].append(year)
                
                gas_volume = gas_volumes[year]
                methane_content = methane_contents[year]
                
                # Calculate engine load for this year
                engine_load, _ = calculate_engine_load(best_engine_size, gas_volume, methane_content)
                
                # Get oil change and maintenance periods based on engine load
                # Find the closest engine load in the data
                load_options = dual_fuel_bg.iloc[:, 0].values
                closest_load = min(load_options, key=lambda x: abs(x - engine_load))
                
                # Get oil change period
                oil_col = None
                for col in dual_fuel_bg.columns:
                    if f'{best_engine_size}' in col and ('Oil change' in col or 'oil' in col.lower()):
                        oil_col = col
                        break
                
                if oil_col is None:
                    raise ValueError(f"Oil change column for engine size {best_engine_size} not found")
                
                oil_change_period = dual_fuel_bg[dual_fuel_bg.iloc[:, 0] == closest_load][oil_col].values[0]
                
                # Get maintenance period
                maint_col = None
                for col in dual_fuel_bg.columns:
                    if f'{best_engine_size}' in col and ('Maintenance' in col or 'maintenance' in col.lower()):
                        maint_col = col
                        break
                
                if maint_col is None:
                    raise ValueError(f"Maintenance period column for engine size {best_engine_size} not found")
                
                maintenance_period = dual_fuel_bg[dual_fuel_bg.iloc[:, 0] == closest_load][maint_col].values[0]
                
                # Calculate oil consumption impacts
                oil_consumption_l = (annual_runtime / oil_change_period) * oil_tank_volume
                for cat in impact_categories:
                    oil_impact_value = dual_fuel_impacts[dual_fuel_impacts['Impact Category'] == 'Operating Resources [1L oil]'][cat].values[0]
                    results['oil_consumption'][cat] += oil_impact_value * oil_consumption_l
                
                # Calculate diesel consumption impacts
                diesel_consumption_l = diesel_consumption * annual_runtime
                for cat in impact_categories:
                    diesel_impact_value = dual_fuel_impacts[dual_fuel_impacts['Impact Category'] == 'Fuel Consumption [1L Diesel]'][cat].values[0]
                    results['diesel_consumption'][cat] += diesel_impact_value * diesel_consumption_l
                
                # Calculate maintenance impacts
                maintenance_factor = annual_runtime / maintenance_period
                for cat in impact_categories:
                    maintenance_impact_value = dual_fuel_impacts[dual_fuel_impacts['Impact Category'] == 'Maintenance [filter production]'][cat].values[0]
                    results['maintenance'][cat] += maintenance_impact_value * maintenance_factor
                
                # Calculate emission impacts
                # Find the closest engine load in emissions data
                emission_load_options = emissions['Engine Load'].values
                closest_emission_load = min(emission_load_options, key=lambda x: abs(x - engine_load))
                
                emission_values = emissions[emissions['Engine Load'] == closest_emission_load].iloc[0, 1:].to_dict()
                
                # Calculate total emissions for the year
                for emission, value in emission_values.items():
                    total_emission = value * gas_volume * annual_runtime
                    
                    # Get impact factors for this emission
                    if emission in emission_impacts['Category'].values:
                        impact_factors = emission_impacts[emission_impacts['Category'] == emission].iloc[0, 1:].to_dict()
                        
                        for cat, factor in impact_factors.items():
                            if cat in results['emissions']:
                                results['emissions'][cat] += total_emission * factor
            
            # Store results for display
            self.results = results
            self.best_engine_size = best_engine_size
            self.gas_df = gas_df
            self.impact_categories = impact_categories
            self.engine_sizes = engine_sizes
            self.selected_engine = selected_engine
            
            # Update display
            self.update_results_tab()
            self.update_details_tab()
            self.update_calculations_tab()
            
            # Switch to results tab
            self.notebook.select(self.results_tab)
            
        except Exception as e:
            messagebox.showerror("Fehler", f"Ein Fehler ist aufgetreten: {str(e)}")
            import traceback
            traceback.print_exc()
    
    def update_results_tab(self):
        # Clear previous results
        for widget in self.results_frame.winfo_children():
            widget.destroy()
        
        # Show selected engine size
        ttk.Label(self.results_frame, text=f"Gewählte Motorgröße: {self.best_engine_size} kW", 
                 font=('Arial', 12, 'bold')).pack(pady=10)
        
        # Create a frame for the chart
        chart_frame = ttk.Frame(self.results_frame)
        chart_frame.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Let user select impact category
        category_frame = ttk.Frame(self.results_frame)
        category_frame.pack(fill='x', padx=10, pady=5)
        
        ttk.Label(category_frame, text="Wirkungskategorie auswählen:").pack(side='left', padx=5)
        category_var = tk.StringVar(value=self.impact_categories[0])
        category_combo = ttk.Combobox(category_frame, textvariable=category_var, values=list(self.impact_categories), state='readonly')
        category_combo.pack(side='left', padx=5)
        category_combo.bind('<<ComboboxSelected>>', lambda e: self.update_chart(chart_frame, category_var.get()))
        
        # Initial chart
        self.update_chart(chart_frame, category_var.get())
        
        # Show total impacts table
        ttk.Label(self.results_frame, text="Gesamtumweltauswirkungen", font=('Arial', 10, 'bold')).pack(pady=10)
        
        # Create a treeview for the table
        tree_frame = ttk.Frame(self.results_frame)
        tree_frame.pack(fill='both', expand=True, padx=10, pady=10)
        
        columns = ('Wirkungskategorie', 'Motorproduktion', 'Ölverbrauch', 'Dieselverbrauch', 'Wartung', 'Emissionen', 'Gesamt')
        tree = ttk.Treeview(tree_frame, columns=columns, show='headings', height=15)
        
        # Define headings
        for col in columns:
            tree.heading(col, text=col)
            tree.column(col, width=120)
        
        # Add data
        for cat in self.impact_categories:
            total = (
                self.results['engine_production'][cat] +
                self.results['oil_consumption'][cat] +
                self.results['diesel_consumption'][cat] +
                self.results['maintenance'][cat] +
                self.results['emissions'][cat]
            )
            
            tree.insert('', 'end', values=(
                cat,
                f"{self.results['engine_production'][cat]:.4f}",
                f"{self.results['oil_consumption'][cat]:.4f}",
                f"{self.results['diesel_consumption'][cat]:.4f}",
                f"{self.results['maintenance'][cat]:.4f}",
                f"{self.results['emissions'][cat]:.4f}",
                f"{total:.4f}"
            ))
        
        # Add scrollbar
        scrollbar = ttk.Scrollbar(tree_frame, orient='vertical', command=tree.yview)
        tree.configure(yscrollcommand=scrollbar.set)
        tree.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
    
    def update_chart(self, parent, category):
        # Clear previous chart
        for widget in parent.winfo_children():
            widget.destroy()
        
        # Prepare data for bar chart
        categories = ['Motorproduktion', 'Ölverbrauch', 'Dieselverbrauch', 'Wartung', 'Emissionen']
        values = [
            self.results['engine_production'][category],
            self.results['oil_consumption'][category],
            self.results['diesel_consumption'][category],
            self.results['maintenance'][category],
            self.results['emissions'][category]
        ]
        
        # Create bar chart
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.bar(categories, values)
        ax.set_ylabel(category)
        ax.set_title(f"Umweltauswirkungen nach Lebensabschnitten - {category}")
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        # Embed chart in tkinter
        canvas = FigureCanvasTkAgg(fig, parent)
        canvas.draw()
        canvas.get_tk_widget().pack(fill='both', expand=True)
    
    def update_details_tab(self):
        # Clear previous details
        for widget in self.details_frame.winfo_children():
            widget.destroy()
        
        # Show gas development
        ttk.Label(self.details_frame, text="Gasentwicklung über die Jahre", font=('Arial', 10, 'bold')).pack(pady=10)
        
        # Create a treeview for the gas data
        tree_frame = ttk.Frame(self.details_frame)
        tree_frame.pack(fill='both', expand=True, padx=10, pady=10)
        
        columns = ('Jahr', 'Gasmenge [m³/h]', 'Methangehalt [%]')
        tree = ttk.Treeview(tree_frame, columns=columns, show='headings', height=10)
        
        # Define headings
        for col in columns:
            tree.heading(col, text=col)
            tree.column(col, width=120)
        
        # Add data
        for _, row in self.gas_df.iterrows():
            tree.insert('', 'end', values=(
                row['Jahr'],
                f"{row['Gasmenge [m³/h]']:.2f}",
                f"{row['Methangehalt [%]']:.2f}"
            ))
        
        # Add scrollbar
        scrollbar = ttk.Scrollbar(tree_frame, orient='vertical', command=tree.yview)
        tree.configure(yscrollcommand=scrollbar.set)
        tree.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        
        # Show engine loads
        ttk.Label(self.details_frame, text="Motorauslastung über die Jahre", font=('Arial', 10, 'bold')).pack(pady=10)
        
        # Calculate engine loads for each year
        engine_loads = []
        for year in range(1, len(self.gas_df) + 1):
            def calculate_engine_load(engine_size, gas_volume, methane_content):
                # Find the correct column name for diesel consumption
                diesel_col = None
                for col in dual_fuel_bg.columns:
                    if f'{engine_size}' in col and ('Diesel' in col or 'diesel' in col.lower()):
                        diesel_col = col
                        break
                
                if diesel_col is None:
                    raise ValueError(f"Diesel consumption column for engine size {engine_size} not found")
                
                diesel_cons = dual_fuel_bg[dual_fuel_bg.iloc[:, 0] == 100][diesel_col].values[0]
                engine_eff = engine_data[engine_data['Engine Size'] == engine_size]['Engine Efficiency'].values[0] / 100
                methane_energy = (methane_content / 100) * gas_volume * calorific_ch4
                diesel_energy = diesel_cons * calorific_diesel
                total_energy = (methane_energy + diesel_energy) * engine_eff
                engine_load = min(100, total_energy / engine_size * 100)
                return engine_load, diesel_cons
            
            gas_volume = self.gas_df.loc[year-1, 'Gasmenge [m³/h]']
            methane_content = self.gas_df.loc[year-1, 'Methangehalt [%]']
            load, _ = calculate_engine_load(self.best_engine_size, gas_volume, methane_content)
            engine_loads.append(load)
        
        load_df = pd.DataFrame({
            'Jahr': range(1, len(self.gas_df) + 1),
            'Motorauslastung [%]': engine_loads
        })
        
        # Create a treeview for the load data
        tree_frame2 = ttk.Frame(self.details_frame)
        tree_frame2.pack(fill='both', expand=True, padx=10, pady=10)
        
        columns2 = ('Jahr', 'Motorauslastung [%]')
        tree2 = ttk.Treeview(tree_frame2, columns=columns2, show='headings', height=10)
        
        # Define headings
        for col in columns2:
            tree2.heading(col, text=col)
            tree2.column(col, width=120)
        
        # Add data
        for _, row in load_df.iterrows():
            tree2.insert('', 'end', values=(
                row['Jahr'],
                f"{row['Motorauslastung [%]']:.2f}"
            ))
        
        # Add scrollbar
        scrollbar2 = ttk.Scrollbar(tree_frame2, orient='vertical', command=tree2.yview)
        tree2.configure(yscrollcommand=scrollbar2.set)
        tree2.pack(side='left', fill='both', expand=True)
        scrollbar2.pack(side='right', fill='y')
    
    def update_calculations_tab(self):
        # Clear previous calculations
        for widget in self.calculations_frame.winfo_children():
            widget.destroy()
        
        # Show engine selection details
        ttk.Label(self.calculations_frame, text="Motorgrößenauswahl", font=('Arial', 10, 'bold')).pack(pady=10)
        
        # Create a treeview for the engine comparison
        tree_frame = ttk.Frame(self.calculations_frame)
        tree_frame.pack(fill='both', expand=True, padx=10, pady=10)
        
        columns = ('Motorgröße [kW]', 'Motorauslastung [%]', 'Dieselverbrauch [L/h]')
        tree = ttk.Treeview(tree_frame, columns=columns, show='headings', height=5)
        
        # Define headings
        for col in columns:
            tree.heading(col, text=col)
            tree.column(col, width=120)
        
        # Add data
        for size in self.engine_sizes:
            def calculate_engine_load(engine_size, gas_volume, methane_content):
                # Find the correct column name for diesel consumption
                diesel_col = None
                for col in dual_fuel_bg.columns:
                    if f'{engine_size}' in col and ('Diesel' in col or 'diesel' in col.lower()):
                        diesel_col = col
                        break
                
                if diesel_col is None:
                    raise ValueError(f"Diesel consumption column for engine size {engine_size} not found")
                
                diesel_cons = dual_fuel_bg[dual_fuel_bg.iloc[:, 0] == 100][diesel_col].values[0]
                engine_eff = engine_data[engine_data['Engine Size'] == engine_size]['Engine Efficiency'].values[0] / 100
                methane_energy = (methane_content / 100) * gas_volume * calorific_ch4
                diesel_energy = diesel_cons * calorific_diesel
                total_energy = (methane_energy + diesel_energy) * engine_eff
                engine_load = min(100, total_energy / engine_size * 100)
                return engine_load, diesel_cons
            
            load, diesel = calculate_engine_load(size, self.initial_gas_volume.get(), self.initial_methane_content.get())
            tree.insert('', 'end', values=(
                size,
                f"{load:.2f}",
                f"{diesel:.2f}"
            ))
        
        # Add scrollbar
        scrollbar = ttk.Scrollbar(tree_frame, orient='vertical', command=tree.yview)
        tree.configure(yscrollcommand=scrollbar.set)
        tree.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        
        # Show constraints for selected engine
        ttk.Label(self.calculations_frame, text="Einschränkungen für gewählten Motor", font=('Arial', 10, 'bold')).pack(pady=10)
        
        # Create a treeview for the constraints
        tree_frame2 = ttk.Frame(self.calculations_frame)
        tree_frame2.pack(fill='both', expand=True, padx=10, pady=10)
        
        columns2 = ('Parameter', 'Wert', 'Einheit')
        tree2 = ttk.Treeview(tree_frame2, columns=columns2, show='headings', height=3)
        
        # Define headings
        for col in columns2:
            tree2.heading(col, text=col)
            tree2.column(col, width=120)
        
        # Add data
        constraints_data = [
            ('Minimaler Gasvolumenstrom', self.selected_engine['Min Volume Flow'].values[0], 'm³/h'),
            ('Maximaler Gasvolumenstrom', self.selected_engine['Max Volume Flow'].values[0], 'm³/h'),
            ('Minimale Motorauslastung', self.selected_engine['Minimum Engine Load'].values[0], '%')
        ]
        
        for data in constraints_data:
            tree2.insert('', 'end', values=data)
        
        # Add scrollbar
        scrollbar2 = ttk.Scrollbar(tree_frame2, orient='vertical', command=tree2.yview)
        tree2.configure(yscrollcommand=scrollbar2.set)
        tree2.pack(side='left', fill='both', expand=True)
        scrollbar2.pack(side='right', fill='y')

# Create and run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = LoCaGasApp(root)
    root.mainloop()

Dual Fuel Background columns: ['Engine Size [kWel]', '250', 'Unnamed: 2', 'Unnamed: 3', '500', 'Unnamed: 5', 'Unnamed: 6', '750', 'Unnamed: 8', 'Unnamed: 9', '1000', 'Unnamed: 11', 'Unnamed: 12']
Dual Fuel Impacts columns: ['Impact Category', 'Fine particulate matter formation ', 'Fossil resource scarcity ', 'Freshwater ecotoxicity', 'Freshwater eutrophication ', 'Global warming', 'Human carcinogenic toxicity ', 'Human non-carcinogenic toxicity -', 'Ionizing radiation ', 'Land use ', 'Marine ecotoxicity ', 'Marine eutrophication ', 'Mineral resource scarcity ', 'Ozone formation, Human health ', 'Ozone formation, Terrestrial ecosystems ', 'Stratospheric ozone depletion ', 'Terrestrial acidification ', 'Terrestrial ecotoxicity ', 'Water consumption ']
Engine Data columns: ['Engine Size', 'Max. volume flow [m^3/h]', 'Min. volume flow [m^3/h]', 'Minimum Engine Load', 'Engine efficency', 'Oil tank volume', 'Production Factor', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unname

Traceback (most recent call last):
  File "/var/folders/yf/7hdsvgn122gd881cyyvpp1400000gn/T/ipykernel_16868/2161735012.py", line 236, in calculate
    engine_load, _ = calculate_engine_load(size, initial_gas_volume, initial_methane_content)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/yf/7hdsvgn122gd881cyyvpp1400000gn/T/ipykernel_16868/2161735012.py", line 209, in calculate_engine_load
    raise ValueError(f"Diesel consumption column for engine size {engine_size} not found")
ValueError: Diesel consumption column for engine size 250 not found
